In [4]:
import sys, os
import numpy as np 
import matplotlib.pyplot as plt
import itertools
import time
import pdb

sys.path.append('..')
from utils import gen_covariance, gen_beta2, gen_data
from utils import selection_accuracy

In [5]:
# Install SLOPE package
from rpy2.robjects.packages import importr
utils = importr('utils')
utils.install_packages('SLOPE')

/home/akumar/anaconda3/envs/nse/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/home/akumar/anaconda3/envs/nse/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
  warnings.warn(x, RRuntimeWarning)
/home/akumar/anaconda3/envs/nse/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The downloaded source packages are in
	‘/tmp/RtmpEx9swm/downloaded_packages’
  warnings.warn(x, RRuntimeWarning)
/home/akumar/anaconda3/envs/nse/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Updating HTML index of packages in '.Library'

  warnings.warn(x, RRuntimeWarning)
/home/akumar/anaconda3/envs/nse/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Making 'packages.html' ...
  warnings.warn(x, RRuntimeWarning)
/home/akumar/anaconda3/envs/nse/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  done


rpy2.rinterface.NULL

In [6]:
# Initialize R interface
slope = importr('SLOPE')

# Autoconvert numpy to R as needed
from rpy2.robjects import numpy2ri
numpy2ri.activate()

In [19]:
# %load ../loaders/datgen.py
from sklearn.preprocessing import StandardScaler

n_features = 50
n_samples = 150

sigma = gen_covariance(n_features, 0, n_features, 1, 0)
beta = gen_beta2(n_features, n_features, sparsity = 0.8, betawidth = np.inf)
X, X_test, y, y_test, ss = gen_data(n_samples, n_features, kappa = 100, 
									covariance = sigma, beta = beta)

X = StandardScaler().fit_transform(X)
X_test = StandardScaler().fit_transform(X_test)

y -= np.mean(y)
y_test -= np.mean(y)

In [15]:
from sklearn.linear_model import lasso_path

In [20]:
# Benchmark against Lasso with coordinate_descent
%time lasso_path(X, y)

CPU times: user 299 ms, sys: 48 µs, total: 299 ms
Wall time: 87 ms


(array([13.42529444, 12.52047453, 11.67663645, 10.88967023, 10.1557429 ,
         9.47127982,  8.83294726,  8.23763618,  7.68244707,  7.16467585,
         6.68180068,  6.23146967,  5.81148947,  5.41981453,  5.05453717,
         4.71387829,  4.39617867,  4.09989094,  3.82357201,  3.56587605,
         3.32554794,  3.10141714,  2.89239201,  2.69745447,  2.51565506,
         2.34610833,  2.18798848,  2.04052538,  1.9030008 ,  1.77474491,
         1.65513305,  1.54358262,  1.43955032,  1.34252944,  1.25204745,
         1.16766364,  1.08896702,  1.01557429,  0.94712798,  0.88329473,
         0.82376362,  0.76824471,  0.71646759,  0.66818007,  0.62314697,
         0.58114895,  0.54198145,  0.50545372,  0.47138783,  0.43961787,
         0.40998909,  0.3823572 ,  0.35658761,  0.33255479,  0.31014171,
         0.2892392 ,  0.26974545,  0.25156551,  0.23461083,  0.21879885,
         0.20405254,  0.19030008,  0.17747449,  0.1655133 ,  0.15435826,
         0.14395503,  0.13425294,  0.12520475,  0.1

In [23]:
%time x = slope.SLOPE(X, y)

CPU times: user 170 ms, sys: 0 ns, total: 170 ms
Wall time: 63.5 ms


In [32]:
np.array(x[4]).shape

(50,)

In [33]:
import pycasso

In [39]:
s1 = pycasso.Solver(X, y, family = 'gaussian', useintercept = False, penalty = 'scad')
s2 = pycasso.Solver(X, y, family = 'gaussian', useintercept = False, penalty = 'mcp')
s3 = pycasso.Solver(X, y, family = 'gaussian', useintercept = False, penalty = 'l1')

In [41]:
%timeit s1.train()

713 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [42]:
%timeit s2.train()

650 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [40]:
%timeit s3.train()

642 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [44]:
os.path.abspath(os.path.join('/home/akumar/nse/uoicorr', '..', 'test.dat'))

'/home/akumar/nse/test.dat'